# Amazon Bedrock Agent - Complete Workflow

This notebook provides a comprehensive workflow to set up and test an Amazon Bedrock agent with the following steps:
- Define resources via CloudFormation.
- Deploy a Knowledge Base and connect it to the agent.
- Test the agent with various scenarios, including using action groups, session attributes, and multi-language queries.
- Validate operations using DynamoDB.


In [ ]:
%store -r stack_name
%store -r unique_id

UNIQUE_ID = unique_id[:8]

stack_name, unique_id

## Environment Setup

Let’s begin by setting up the environment and configuring the necessary AWS resources. This step ensures that the foundation is ready for deploying and running the application seamlessly.

In [ ]:
from dotenv import load_dotenv
import botocore.exceptions
import os
import json
import sys
import boto3
import pprint as pp
import time
import uuid

aws_region = "us-east-1"
load_dotenv(".env")

def interactive_sleep(seconds):
    """Sleep interactively for the given number of seconds with progress."""
    for i in range(seconds):
        print(f"Sleeping... {i + 1}/{seconds} seconds", end="\r")
        time.sleep(1)

AGENT_NAME = os.getenv("AGENT_NAME", 'booking-agent') 
VECTOR_INDEX_NAME = os.getenv("VECTOR_INDEX_NAME", f'{AGENT_NAME}-kb-{UNIQUE_ID}')

bedrock_runtime_client = boto3.client("bedrock-runtime", region_name=aws_region)
bedrock_management_client = boto3.client('bedrock', region_name=aws_region)
bedrock_agent_client = boto3.client('bedrock-agent', region_name=aws_region)
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name=aws_region)
cloudformation_client = boto3.client('cloudformation', region_name=aws_region)

boto3.__version__

In [ ]:
response = cloudformation_client.describe_stacks(StackName=stack_name)
outputs = response['Stacks'][0]['Outputs']

lab2_results = {}
for output in outputs:
    lab2_results[output['OutputKey']] = output['OutputValue']
    print(f"{output['OutputKey']}: {output['OutputValue']}")

## Creating an Index for Booking in OpenSearch

To enable the agent to interact with the required knowledge base (KB), we must first create an index in the OpenSearch database. Without this index, the KB cannot be established, and the agent will lack the necessary data for its operations.

In [ ]:
import os
from opensearch_utils import create_index

HOST = lab2_results["OpenSearchServerlessCollectionEndpoint"].replace('https://', '')

try:
    print("Starting OpenSearch index creation...")
    create_index(host=HOST, index_name=VECTOR_INDEX_NAME, region=aws_region)
    print("OpenSearch index creation completed successfully.")
except Exception as e:
    print(f"Error during OpenSearch index creation: {str(e)}")

## Loading a CloudFormation Template

A **CloudFormation template** is a blueprint that defines the infrastructure and resources required for your application.

In [ ]:
from cloudformation_utils import load_template, colorize_yaml

template_file_path = 'bedrock_agent_template.yaml'

In [ ]:
try:
    cloudformation_template = load_template(template_file_path)
    colorful_yaml = colorize_yaml(cloudformation_template)
    print(colorful_yaml)
except Exception as e:
    print(f"An error occurred: {e}")

### Create Resources via CloudFormation

We will use a pre-defined CloudFormation template to deploy resources, including:
- Amazon S3 Bucket for Knowledge Base documents.
- Amazon DynamoDB Table for restaurant bookings.
- Amazon OpenSearch Serverless collection for vector search.
- Amazon Bedrock Knowledge Base and related resources.

**Instructions:** Before proceeding, ensure that your AWS account has the necessary permissions to deploy the CloudFormation stack.

In [ ]:
agent_stack_name = f'BedrockAgentsStack-{UNIQUE_ID}'
parameters = [
    {
        'ParameterKey': 'BedrockExecutionRoleArn',
        'ParameterValue': lab2_results['BedrockExecutionRoleArn']
    },
    {
        'ParameterKey': 'VectorIndexName',
        'ParameterValue': VECTOR_INDEX_NAME
    },
    {
        'ParameterKey': 'KnowledgeBaseId',
        'ParameterValue': lab2_results['BedrockKnowledgeBaseId']
    },
    {
        'ParameterKey': 'S3BucketName',
        'ParameterValue': lab2_results['S3BucketName']
    },
    {
        'ParameterKey': 'OpenSearchCollectionArn',
        'ParameterValue': lab2_results['OpenSearchServerlessCollectionArn']
    },
    {
        'ParameterKey': 'CreateKnowledgeBase',
        'ParameterValue': 'true'
    },
    {
        'ParameterKey': 'UUID',
        'ParameterValue': UNIQUE_ID
    },
    {
        'ParameterKey': 'FoundationModelId',
        'ParameterValue': "amazon.titan-text-premier-v1:0"
    }
]

In [ ]:
from cloudformation_utils import create_stack

%store agent_stack_name

try:
    cloudformation_template = load_template(template_file_path)
    stack_id = create_stack(aws_region, agent_stack_name, cloudformation_template, parameters)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

### Waiting for CloudFormation Stack Completion

Before proceeding to the next steps, it's essential to wait for the **CloudFormation stack** to complete its creation process. This ensures that all resources are provisioned and configured correctly.

In [ ]:
from cloudformation_utils import wait_for_stack

wait_for_stack(aws_region, agent_stack_name, 'CREATE_COMPLETE')

### (Optional) Experiment with Different Models and Update the Template

To explore alternative configurations, you can experiment with different models and update the template accordingly. This allows you to optimize the setup and align it with your specific requirements.

In [ ]:
import ipywidgets as widgets

agent_foundation_model_selector = widgets.Dropdown(
    options=[
        (model['modelName'], model['modelId']) 
        for model in bedrock_management_client.list_foundation_models(
            byOutputModality="TEXT",
            byInferenceType="ON_DEMAND"
        ).get('modelSummaries', [])  if model['providerName'] in ('Amazon', 'Anthropic') 
    ],
    value='anthropic.claude-3-sonnet-20240229-v1:0',
    description='FM:',
    disabled=False,
)
agent_foundation_model_selector

In [ ]:
from cloudformation_utils import update_parameters, update_stack
from cloudformation_utils import wait_for_stack

try:
    parameters = update_parameters(parameters, key="FoundationModelId", value=agent_foundation_model_selector.value)
    cloudformation_template = load_template(template_file_path)
    stack_id = update_stack(aws_region, agent_stack_name, cloudformation_template, parameters)
    wait_for_stack(aws_region, stack_name, 'UPDATE_COMPLETE')
except Exception as e:
    print(f"An unexpected error occurred: {e}")

### Retrieve Stack Outputs

Get the outputs from the stack, such as the DynamoDB table name and IAM role ARN, to use them in subsequent steps.

In [ ]:
response = cloudformation_client.describe_stacks(StackName=agent_stack_name)
outputs = response['Stacks'][0]['Outputs']

lab4_results = {}
for output in outputs:
    lab4_results[output['OutputKey']] = output['OutputValue']
    print(f"{output['OutputKey']}: {output['OutputValue']}")

## Upload Knowledge Base Documents to S3

The next step involves uploading the Knowledge Base (KB) documents to the **S3 bucket** provisioned by the CloudFormation stack. This ensures that the KB is securely stored and accessible for processing.

In [ ]:
import os
import boto3

s3_client = boto3.client('s3')
bucket_name = lab2_results['S3BucketName']
document_path = "kb_restaurant"


def upload_directory(path, bucket_name, prefix):
    for root, dirs, files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root, file)
            relative_path = os.path.relpath(file_to_upload, path)
            s3_key = f"{prefix}/{relative_path.replace(os.sep, '/')}" 
            print(f"Uploading {file_to_upload} to S3 bucket {bucket_name} with key {s3_key}")
            s3_client.upload_file(file_to_upload, bucket_name, s3_key)

            
upload_directory(document_path, bucket_name, prefix=document_path)

### Sync the Knowledge Base

Once the documents are uploaded, synchronize the Knowledge Base to ensure the latest data is accessible to the system.

In [ ]:
def synchronize_data(bedrock_client, kb_id, ds_id):
    try:
        start_job_response = bedrock_client.start_ingestion_job(
            knowledgeBaseId=kb_id,
            dataSourceId=ds_id
        )
        job = start_job_response["ingestionJob"]
        print(f"Started ingestion job: {job['ingestionJobId']}")

        while job['status'] not in ['COMPLETE', 'FAILED']:
            job_status_response = bedrock_client.get_ingestion_job(
                knowledgeBaseId=kb_id,
                dataSourceId=ds_id,
                ingestionJobId=job["ingestionJobId"]
            )
            job = job_status_response["ingestionJob"]
            print(f"Ingestion job status: {job['status']}")
            if job['status'] == 'FAILED':
                raise Exception(f"Ingestion job failed with reason: {job.get('failureReason')}")
            time.sleep(30)

        print(f"Ingestion job completed with status: {job['status']}")
    except ClientError as e:
        print(f"Error occurred during synchronization: {e}")
        raise

**Instructions:**

1. **Extract IDs:**
   - Retrieve the `BedrockKnowledgeBaseId` from `lab4_results`.
   - Parse the `BedrockDataSourceId` and extract the second part after splitting it by the delimiter `"|"`.

2. **Start Synchronization:**
   - Use the `synchronize_data` function to initiate the sync job, passing the extracted Knowledge Base and Data Source IDs along with the `bedrock_agent_client`.

<details>
<summary>Click here for the solution</summary>

```python
# Extract the Knowledge Base ID
kb_id = lab4_results["BedrockKnowledgeBaseId"]

# Extract the Data Source ID (split and take the second part)
ds_id = lab4_results["BedrockDataSourceId"].split("|")[1]

# Start the synchronization process
synchronize_data(bedrock_agent_client, kb_id, ds_id)
```

</details>

## Testing the Agent Invocation

In this step, we will invoke the **Bedrock agent** and test its responses to various queries. Before proceeding, ensure that the `agent_id` and `alias_id` are correctly retrieved from the CloudFormation stack outputs.

In [ ]:
import uuid
import boto3
import pprint

pp = pprint.PrettyPrinter(indent=2)


def invoke_agent_helper(query, session_id, agent_id, alias_id, enable_trace=False, session_state=None):
    """
    Invokes the Bedrock agent and streams responses.

    Args:
        query (str): The input text query for the agent.
        session_id (str): Unique identifier for the session.
        agent_id (str): ID of the agent to be invoked.
        alias_id (str): Alias of the agent version (e.g., 'DRAFT').
        enable_trace (bool, optional): Enable trace for debugging purposes.
        session_state (dict, optional): State of the session to continue a conversation.

    Returns:
        str: The final response from the agent.
    """
    if not session_state:
        session_state = {}

    payload = {
        "inputText": query,
        "agentId": agent_id,
        "agentAliasId": alias_id,
        "sessionId": session_id,
        "enableTrace": enable_trace,
        "endSession": False,
        "sessionState": session_state
    }

    try:
        agent_response = bedrock_agent_runtime_client.invoke_agent(**payload)
        event_stream = agent_response['completion']
        for event in event_stream:
            if 'chunk' in event:
                data = event['chunk']['bytes']
                response_text = data.decode('utf8')
                return response_text
            elif 'trace' in event and enable_trace:
                pp.pprint(event['trace'])
            else:
                raise Exception(f"Unexpected event received: {event}")

    except Exception as e:
        print(f"Error while invoking agent: {e}")
        raise

### Testing the Agent and its Functionalities

To ensure that the agent behaves as expected, we will perform a series of tests in a step-by-step manner. This involves verifying the agent's invocation without a knowledge base (KB), associating a KB with the agent, and testing the agent's ability to perform actions before and after associating an action group.

**Instructions:**

1. **List Agent Aliases:**
   - Use the AWS CLI command to list the available aliases for the agent specified in `lab4_results["BedrockAgentId"]`.

2. **Extract Required IDs:**
   - Retrieve the `BedrockAgentId` from `new_results`.
   - Read the output from the AWS CLI command to determine the `alias_id` and `agent_version`.
   - Assign the `alias_id` and `agent_version` based on the CLI output.

3. **Verify IDs:**
   - Print or log the extracted `agent_id`, `alias_id`, and `agent_version` to confirm they are correct before using them in further operations.

<details>
<summary>Click here for the solution</summary>

```python
# List agent aliases using AWS CLI
!aws bedrock-agent list-agent-aliases --agent-id {lab4_results["BedrockAgentId"]}

# Extract the agent ID from results
agent_id = lab4_results["BedrockAgentId"]

# Manually assign or retrieve alias ID and agent version based on CLI output
alias_id = "read-from-cli-output"
agent_version = "read-from-cli-output"

# Print the extracted values for verification
print(f"Agent ID: {agent_id}")
print(f"Alias ID: {alias_id}")
print(f"Agent Version: {agent_version}")
```

</details>

#### Test Agent Invocation Without a Knowledge Base

Initially, we will test the agent's response to queries without any associated Knowledge Base (KB). This will help us verify the agent's default behavior and foundational functionality when no external data sources are available.

In [ ]:
session_id:str = str(uuid.uuid1())

In [ ]:
%%time
query = "What is in the children's menu?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print("Agent response:", response)

### Associate a Knowledge Base with the Agent

To enhance the agent's capabilities, we will associate a Knowledge Base (KB) containing restaurant booking information. This step uses the [`AssociateAgentKnowledgeBase`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/associate_agent_knowledge_base.html) API in boto3. Associating a KB allows the agent to query specific information stored in the KB, improving its responses to user queries.

To enable the agent to utilize the Knowledge Base (KB) effectively, you need to associate the KB with the agent. This step allows the agent to access the KB for answering relevant queries.

#### Required Parameters:

- **`agentId`**: The ID of the agent to which the KB will be associated.
- **`agentVersion`**: The version of the agent (e.g., `'DRAFT'` or `'ACTIVE'`).
- **`knowledgeBaseId`**: The ID of the Knowledge Base to be linked to the agent.
- **`knowledgeBaseState`**: Set to `'ENABLED'` to activate the KB for use.
- **`description`**: A brief explanation of how the KB will be utilized, e.g., `'Access the knowledge base when customers ask about the plates in the menu.'`

<details>
<summary>Click here for the solution</summary>

```python
description = "Access the knowledge base when customers ask about the plates in the menu."
response = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion=agent_version,
    description=description,
    knowledgeBaseId=kb_id,
    knowledgeBaseState='ENABLED'
)
```
</details>

### Preparing the Agent for Activation

After making updates to the agent, it is necessary to transition its state from **DRAFT** to **ACTIVE**. This ensures the agent is fully operational and ready for production use.

In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)

### Test Agent Invocation With the Knowledge Base

After associating the KB, we will test the agent again to validate that it is successfully utilizing the KB for answering queries. This ensures that the integration between the agent and the KB is functioning correctly.

In [ ]:
session_id:str = str(uuid.uuid1())

In [ ]:
%%time
query = "What is in the children's menu?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print("Agent response:", response)

#### Test the Agent Performing an Action Without Action Group Association

Before associating an action group, we will test the agent's ability to perform defined actions, such as creating or retrieving a booking. This step ensures that the agent correctly reports any missing or unlinked functionalities, highlighting the need for an associated action group.

In [ ]:
session_id:str = str(uuid.uuid1())

In [ ]:
%%time
query = "Hi, I am Anna. I want to create a booking for 2 people, at 8pm on the 5th of May 2024."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print("Agent response:", response)

### Defining and Associating an Agent Action Group

With the Lambda function now created, the next step is to define an [Action Group](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-create.html) and link it to the agent. Action Groups enable the agent to perform specific tasks—in this case, handling booking operations. 

To achieve this, we provide the agent with a [function schema](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-function.html) in `JSON` format. This schema describes the capabilities of the Lambda function, including the function name, a brief description of its purpose, and details about its parameters.

Each parameter must include:
- **Name**: The identifier of the parameter.
- **Description**: A clear explanation of its role.
- **Type**: The expected data type for the parameter.
- **Required**: A flag indicating whether the parameter is mandatory for the function.

The schema for these functions will be defined as `agent_functions` in JSON format. This structure allows the agent to understand and execute its associated tasks effectively.

In [ ]:
agent_functions = [
    {
        'name': 'get_booking_details',
        'description': 'Retrieve details of a restaurant booking',
        'parameters': {
            "booking_id": {
                "description": "The ID of the booking to retrieve",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'create_booking',
        'description': 'Create a new restaurant booking',
        'parameters': {
            "date": {
                "description": "The date of the booking in the format YYYY-MM-DD",
                "required": True,
                "type": "string"
            },
            "name": {
                "description": "Name to idenfity your reservation",
                "required": True,
                "type": "string"
            },
            "hour": {
                "description": "The hour of the booking in the format HH:MM",
                "required": True,
                "type": "string"
            },
            "num_guests": {
                "description": "The number of guests for the booking",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'delete_booking',
        'description': 'Delete an existing restaurant booking',
        'parameters': {
            "booking_id": {
                "description": "The ID of the booking to delete",
                "required": True,
                "type": "string"
            }
        }
    },
]

### Attaching the Action Group

The following Python code executes the creation of the Action Group using the Bedrock Agent SDK. This ensures the Action Group is properly registered and associated with the agent.

In [ ]:
agent_action_group_name = "TableBookingActions"
agent_action_group_description = "Provides functions for booking-related tasks such as retrieving, creating, or deleting a booking."
action_group_executor = {'lambda': lab4_results['LambdaFunctionArn']}
function_schema = {'functions': agent_functions}

To enhance the agent's capabilities, use the `bedrock_agent_client.create_agent_action_group` API to attach an action group. This allows the agent to execute specific actions as defined in the action group.

### Required Parameters:

- **`agentId`**: The ID of the agent to which the action group will be attached.
- **`agentVersion`**: The version of the agent (e.g., `'DRAFT'`).
- **`actionGroupExecutor`**: Specifies the executor that handles the actions within the group.
- **`actionGroupName`**: The name of the action group being created.
- **`functionSchema`**: Defines the schema for the actions in the group.
- **`description`**: A brief explanation of the action group’s purpose and functionality.

<details>
<summary>Click here for the solution</summary>

```python
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor=action_group_executor,
    actionGroupName=agent_action_group_name,
    functionSchema=function_schema,
    description=agent_action_group_description
)
```

</details>

### Preparing the Agent for Activation

After making updates to the agent, it is necessary to transition its state from **DRAFT** to **ACTIVE**. This ensures the agent is fully operational and ready for production use.

In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)

#### Test the Agent Performing an Action After Action Group Association

Finally, we will test the agent's ability to execute actions after associating the action group. This includes verifying functionalities like creating, retrieving, and deleting bookings. A successful test confirms that the agent is now fully functional and capable of performing its intended tasks.


In [ ]:
session_id:str = str(uuid.uuid1())

In [ ]:
%%time
query = "Hi, I am Anna. I want to create a booking for 2 people, at 8pm on the 5th of May 2024."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print("Agent response:", response)

#### Validate DynamoDB Table

We will verify that restaurant booking operations (e.g., creating a reservation) are correctly reflected in the DynamoDB table.

In [ ]:
dynamodb = boto3.resource('dynamodb')
table_name = lab4_results['DynamoDBTableName']
table = dynamodb.Table(table_name)

response = table.scan()
items = response['Items']
print("DynamoDB table items:", json.dumps(items, indent=4))

## Agents for Amazon Bedrock: Testing Agent Invocation

This section provides an overview of testing the **Amazon Bedrock agent invocation** using the [`boto3`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime.html) client for `bedrock-agent-runtime`. After creating the agent and associating the Knowledge Base (KB), the next step is to test its functionality by invoking it with various queries.

#### What You’ll Do:
- **Invoke the Agent with a New Session:** Use the Knowledge Base and Action Groups to execute queries in a fresh session.
- **Invoke the Agent with an Existing Session:** Continue interactions within an established session to preserve context.
- **Invoke the Agent with Prompt Attributes:** Customize the invocation with additional parameters and attributes.
- **Enable Tracing for Debugging:** Activate trace logging to monitor the agent's decision-making process.
- **Invoke the Agent in Different Languages:** Test the agent's multilingual capabilities by sending queries in various languages.

We use a helper function, `invoke_agent_helper`, to interact with the agent in a flexible manner. This function lets users send a `query` to the agent while optionally enabling trace functionality and maintaining session context.

### Key Features:

1. **Session Management:**
   - The `session_id` parameter allows users to manage conversation sessions:
     - Providing a new `session_id` starts a fresh conversation without prior context.
     - Reusing an existing `session_id` continues the conversation with previously shared context.

2. **Trace Option:**
   - When the `enable_trace` parameter is set to `True`, the agent’s responses include detailed trace information. This trace outlines the reasoning or logical steps (e.g., Chain of Thoughts prompting) that the agent followed to arrive at the given response, providing insights into the agent's decision-making process.

3. **Session State:**
   - The `session_state` parameter accepts a dictionary that allows for the sharing of specific session-related information. It includes:
     - **`sessionAttributes`**: These attributes persist across all calls with the same `session_id` within a session, as long as the session remains active. While these attributes are accessible to the agent's Lambda function, they are not included in the agent's prompt directly. For example:
       - You can implement fine-grained API access control using Lambda integration.
       - Refer to [this example](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-for-bedrock/features-examples/09-fine-grained-access-permissions) for further details.
     - **`promptSessionAttributes`**: These attributes are specific to a single `invoke_agent_helper` call. They are included in both the agent's prompt and Lambda function. The `$prompt_session_attributes$` placeholder can be used when defining the orchestration base prompt.
     - **`invocationId`**: Used to pass the `invocationId` from a previous control payload, enabling you to provide additional context or responses for a Return of Control action. More details can be found [here](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-for-bedrock/features-examples/03-create-agent-with-return-of-control).
     - **`returnControlInvocationResults`**: Contains the results of actions executed outside the agent. This field is required for Return of Control scenarios, as explained in [this example](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-for-bedrock/features-examples/03-create-agent-with-return-of-control).

#### Create support selectAllFromDynamoDB function

We will also create the support function called `selectAllFromDynamoDB` to select all data in the dynamoDB table `restaurant_bookings`. This function will be used to validate our agent's behaviour

In [ ]:
import logging
import pprint
import json
import pandas as pd

dynamodb = boto3.resource('dynamodb')

def selectAllFromDynamodb():
    table = dynamodb.Table(table_name)
    
    response = table.scan()
    items = response['Items']

    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        items.extend(response['Items'])

    items = pd.DataFrame(items)
    return items

In [ ]:
items = selectAllFromDynamodb()
items

### Invoking agent with a new session id
Let's first use the `InvokeAgent` function to query the Knowledge Base with the agent without previous context

In [ ]:
session_id:str = str(uuid.uuid1())

In [ ]:
%%time
query = "What is in the childrens menu?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

### Invoking agent with existent session id

Next we can use the context to ask a follow up question. To do so, we use the same `session_id` with the `invokeAgent` function

In [ ]:
%%time
query = "Which of those options are vegetarian?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

As you can see, the agent knows that we are talking about the children's menu.

### Invoking agent using action group

Now let's use our agent to make a reservation. By doing so, we will require the agent to execute an action from our action group to create a new reservation.

In [ ]:
%%time
query = "Hi, I am Maria. I want to create a booking for 4 people, at 9pm on the 5th of May 2024."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

Let's double check that the data was properly added to the dynamoDB table

In [ ]:
selectAllFromDynamodb()

When making restaurant reservations with our agent, it’s common to already be logged into a system that knows our name. Wouldn't it be fantastic if the agent could use that information too?

We can achieve this by utilizing the session context to pass attributes directly into the agent's prompt. Specifically, we'll use the [`promptSessionAttributes`](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-session-state.html) parameter to include these details. Additionally, we’ll start a fresh session with a new `session_id` to ensure the agent doesn’t retain prior knowledge of our name.

In [ ]:
session_id:str = str(uuid.uuid1())

In [ ]:
%%time
query = "I want to create a booking for 2 people, at 8pm on the 6th of May 2024."
session_state = {
    "promptSessionAttributes": {
        "name": "John"
    }
}
response = invoke_agent_helper(query, session_id, agent_id, alias_id, session_state=session_state)
print(response)

Again let's validate the the correct data was added to dynamoDB

In [ ]:
selectAllFromDynamodb()

We can also validate the data using our agent's session information since the agent knows the booking id to invoke the get booking details function

In [ ]:
%%time
query = "Get the details for the last booking created"
booking_id = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(booking_id)

#### Deleting booking created

Let's also test the delete booking functionality by deleting the last created booking id

In [ ]:
%%time
query = f"I want to delete the booking."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

And we confirm that the booking has also been deleted from the dynamoDB table

In [ ]:
selectAllFromDynamodb()

#### Invoking agent using promptSessionAttributes to handle temporal information

With real-life applications, context is really important. We want to make reservations considering the current date and the days sorounding it. Agents for Amazon Bedrock also allow you to provide temporal context for the agent with the prompt attributes. Let's test it with a reservation for tomorrow

In [ ]:
query = "I want to create a booking for 2 people, at 8pm tomorrow."

Using `promptSessionAttributes` to Provide Today's Date in `'MMM-DD-YYYY'` Format

Use the `invoke_agent_helper` function with `promptSessionAttributes` to pass the current date to the agent. The date will be formatted as `'MMM-DD-YYYY'` (e.g., `Dec-08-2024`). Additionally, we'll make a booking request for tomorrow using this information using the above `query`.

<details>
<summary>Click here to view the solution</summary>

```python
%%time
# Importing the datetime module to retrieve today's date
from datetime import datetime

# Format today's date as 'MMM-DD-YYYY'
today = datetime.today().strftime('%b-%d-%Y')

# Creating a session ID and making a booking request for tomorrow
session_id: str = str(uuid.uuid1())
query = "I want to create a booking for 2 people, at 8pm tomorrow."
session_state = {
    "promptSessionAttributes": {
        "name": "John",  # User's name
        "today": today   # Current date in specified format
    }
}

# Invoking the agent with the query and session context
response = invoke_agent_helper(query, session_id, agent_id, alias_id, session_state=session_state)
print(response)
```

</details>

And now to confirm that everything got added properly, let's retrieve the details of the last booking 

In [ ]:
%%time
query = "Could you get the details for the last booking created?"
booking_id = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(booking_id)

#### Asking the agent for food recomendations

Another good use case for our agent, is to use its reasoning capabilities to ask for some food recommendation. Let's look at a couple of examples for it

In [ ]:
%%time
session_id:str = str(uuid.uuid1())
query = "What do you have for kids that don't like fries?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())
query = "I am allergic to shrimps. What can I eat at this restaurant?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

### Invoking agent with trace enabled

We can also invoke our agent with the trace enabled to produce the details of each step being orchestrated by the Agent. Let's see all the steps executed when checking for documents in the knowledge base

In [ ]:
%%time
session_id:str = str(uuid.uuid1())
query = "What are the desserts on the adult menu?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id, enable_trace=True)
print(response)

### Invoking agent with different prompt languages

The last feature that we will highlight in this notebook is the ability of LLM models to handle inputs in different languages. We've only implemented our agent in English, but wouldn't it be great if it could also handle other languages?

Good news is that it can! This is due to the LLM multi-language capabilities. And the best part is that we don't need to change anything in the agent and it will even going to respond in the requested language.

Let's try a couple of queries!

In [ ]:
%%time
# Invoking agents in spanish
session_id:str = str(uuid.uuid1())
query = "¿Podrías reservar una mesa para dos 25/07/2024 a las 19:30"
session_state = {
    "promptSessionAttributes": {
        "Nombre": "Gabriela"
    }
}
response = invoke_agent_helper(query, session_id, agent_id, alias_id, session_state=session_state)
print(response)

In [ ]:
%%time
# Invoking agents in german
session_id:str = str(uuid.uuid1())
query = "Könnten Sie heute Abend einen Tisch für zwei reservieren? Um 19:30 Uhr"
session_state = {
    "promptSessionAttributes": {
        "Name": "Julian",
        "Heute": today
    }
}
response = invoke_agent_helper(query, session_id, agent_id, alias_id, session_state=session_state)
print(response)

Last, let's check our dynamoDB to see all the bookings available

In [ ]:
selectAllFromDynamodb()

### Interact with the Agent on Topics Unrelated to Restaurant Booking or Menus

This section outlines the constraints and boundaries (guardrails) that ensure the agent operates effectively and stays within the intended scope of the interaction. These guardrails are designed to:

- Prevent the agent from straying into unrelated domains or topics.
- Maintain the focus of the conversation while ensuring high-quality, relevant responses.
- Handle queries outside the primary scope gracefully, providing appropriate fallback responses or redirection. 

These measures ensure the agent remains user-friendly and contextually accurate, even when dealing with topics unrelated to restaurant bookings or menus.

In [ ]:
query = """Use only your built-in knowledge from the training set. 
Who discovered gravity, and how did it influence the development of modern physics?"""

In [ ]:
%%time
session_id:str = str(uuid.uuid1())
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

### Using AWS CLI to Retrieve Guardrail Identifier and Version for `update_agent`

To perform the `update_agent` operation, you need to obtain the `guardrailIdentifier` and `guardrailVersion`. Use the AWS CLI to fetch this information efficiently.

In [ ]:
!aws bedrock get-guardrail --guardrail-identifier {lab4_results['GuardrailId']}

In [ ]:
agent_description = bedrock_agent_client.get_agent(agentId=agent_id)['agent']
response = bedrock_agent_client.update_agent(
    guardrailConfiguration={
        'guardrailIdentifier': "<your-solution-here>,
        'guardrailVersion': "<your-solution-here>
    },
    agentId=agent_description['agentId'],
    agentName=agent_description['agentName'],
    agentResourceRoleArn=agent_description['agentResourceRoleArn'],
    description=agent_description['description'],
    foundationModel=agent_description['foundationModel'],
    idleSessionTTLInSeconds=agent_description['idleSessionTTLInSeconds'],
    instruction=agent_description['instruction']
)

Each time an update is made, you must prepare the agent to ensure the changes are successfully implemented and go live.

In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

### **Challenge 1: Implement Multi-Language Support with Knowledge Base Queries**

**Objective:** Expand the agent's capability to handle multi-language queries by leveraging the Knowledge Base and LLM's multilingual capabilities.

#### **Scenario:**
Your agent manages a restaurant booking system but needs to support users in different languages, such as **French** and **Italian**, while maintaining access to the Knowledge Base.

#### **Tasks:**

1. **Enable Multilingual Support:**
   - Use the same Knowledge Base but query it in French and Italian.
   - Ensure the agent responds in the query language.
   - Examples:
     - *French:* "Quelles sont les options végétariennes dans le menu enfant?"
     - *Italian:* "Quali opzioni vegetariane ci sono nel menu per bambini?"

2. **Test Knowledge Base Integration:**
   - Verify that the Knowledge Base queries return accurate information regardless of the query language.
   - Ensure the responses maintain language consistency.

3. **Handle Context with Session Attributes:**
   - Use **promptSessionAttributes** to add session-specific context in the user's language.
   - Example:
     - French: Add "nom" (name) as a session attribute for personalized responses.
     - Italian: Add "oggi" (today) as a temporal session attribute.

4. **Evaluate Response Accuracy:**
   - Test with ambiguous or partially translated queries.
   - Examples:
     - "I want to book a table pour deux personnes."
     - "Cosa c'è nel menu per bambini with vegetarian options?"

#### **Deliverable:**
- Python code to invoke multilingual queries.
- A JSON file containing agent responses in different languages.
- A brief report discussing the accuracy and consistency of responses, and any limitations observed in handling multi-language queries.
- **How well do different models handle various languages?**

### **Challenge 2: Using Prompt Engineering to Navigate Guardrails**

#### **Objective:**
Test the robustness of the guardrail system created in **Challenge 1** by leveraging **prompt engineering** to craft queries that attempt to bypass or exploit its logic while still adhering to the input constraints.

#### **Scenario:**
The restaurant booking agent is designed to block queries involving sensitive or restricted information, such as credit card details or customer personal data. Your task is to craft carefully engineered prompts to determine if the guardrails can be bypassed, while maintaining the appearance of valid queries.

#### **Tasks:**

1. **Analyze the Guardrail Rules:**
   - Review the JSON schema and logic used for the guardrail.
   - Identify potential weaknesses or gaps in how the rules are applied (e.g., synonyms, ambiguous phrasing).

2. **Craft Prompt Engineering Queries:**
   - Write queries that attempt to bypass the guardrail while appearing valid or indirect. 
   - Attempt queries in creative ways, such as:
     - Using synonyms or alternative phrasing.
     - Splitting sensitive requests across multiple queries to disguise intent.

3. **Test the Guardrail:**
   - Query the agent with a combination of:
     - *Valid queries*: Designed to retrieve legitimate booking information.
     - *Engineered bypass queries*: Designed to test the guardrail’s robustness.

4. **Evaluate Guardrail Robustness:**
   - Record whether each query was successfully blocked by the guardrail or allowed through.
   - Identify patterns or weaknesses in the guardrail's logic.

#### **Deliverables:**
- Python Code to test the guardrail with both valid and engineered queries.
- A list containing the tested queries.
- Discuss the effectiveness of the guardrail in handling prompt engineering attempts.
- **How effectively do different models implement guardrails?**